In [ ]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np
import scipy
from scipy.special import gamma
from scipy.special import eval_legendre

# Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly

# Atoms
import ase
from ase.io import read
from rascal.representations import SphericalInvariants
from rascal.neighbourlist.structure_manager import AtomsList
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species

# Utilities
import h5py
import json
from selection import FPS, random_selection
from project_utils import load_structures_from_hdf5

# SOAP
from soap import quippy_soap, librascal_soap

# Simple structure

In [ ]:
a = 3
frames = [ase.Atoms('CsCl', 
                    positions=[[0, 0, 0], 
                               [a/2, a/2, a/2]],
                    cell=[[a, 0, 0],
                          [0, a, 0],
                          [0, 0, a]],
                    pbc=[1, 1, 1])]

# GTO power spectrum

In [ ]:
def GTO_sigma(cutoff, n, n_max):
    """
        Compute GTO sigma
    """
    return np.maximum(np.sqrt(n), 1) * cutoff / n_max

def GTO_width(sigma):
    """
        Compute GTO width
    """
    return 1.0 / (2 * sigma ** 2)

def GTO_prefactor(n, sigma):
    """
        Compute GTO prefactor
    """
    return np.sqrt(2 / (sigma ** (2 * n + 3) * gamma(n + 1.5)))

def GTO(r, n, sigma):
    """
        Compute GTO
    """
    b = GTO_width(sigma)
    N = GTO_prefactor(n, sigma)
    return N * r ** (n + 1) * np.exp(-b * r ** 2) # why n+1?

def GTO_overlap(n, m, sigma_n, sigma_m):
    """
        Compute overlap of two GTOs
    """
    b_n = GTO_width(sigma_n)
    b_m = GTO_width(sigma_m)
    N_n = GTO_prefactor(n, sigma_n)
    N_m = GTO_prefactor(m, sigma_m)
    nm = 0.5 * (3 + n + m)
    return 0.5 * N_n * N_m * (b_n + b_m) ** (-nm) * gamma(nm) # why 0.5?

def legendre_polynomials(l, x):
    """
        Evaluate Legendre Polynomials
    """
    return eval_legendre(l, x)

In [ ]:
n_max = 10
l_max = 4
cutoff = 6

hypers = {
    "soap_type": "PowerSpectrum",
    "radial_basis": "GTO",
    "interaction_cutoff": cutoff,
    "max_radial": n_max,
    "max_angular": l_max,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "normalize": False
}
r_grid = np.linspace(0, cutoff, 50)
t_grid = np.linspace(-1, 1, 50)

representation = SphericalInvariants(**hypers)
soaps = representation.transform(frames[0]).get_features(representation)
atom_list = AtomsList(frames[0], representation.nl_options)
feature_idx_map = representation.get_feature_index_mapping(atom_list)

In [ ]:
dr = np.diff(r_grid)[0]
dt = np.diff(t_grid)[0]

In [ ]:
n_grid, m_grid = np.meshgrid(np.arange(0, n_max), np.arange(0, n_max))
n_grid = n_grid.T
m_grid = m_grid.T

In [ ]:
sigma_n_grid = GTO_sigma(cutoff, n_grid, n_max)
sigma_m_grid = GTO_sigma(cutoff, m_grid, n_max)

In [ ]:
S = GTO_overlap(n_grid, m_grid, sigma_n_grid, sigma_m_grid)

In [ ]:
S_inv = scipy.linalg.fractional_matrix_power(S, -0.5)

In [ ]:
nr_grid, rr_grid = np.meshgrid(np.arange(0, n_max), r_grid)
nr_grid = nr_grid.T
rr_grid = rr_grid.T

sigma_nr_grid = GTO_sigma(cutoff, nr_grid, n_max)

In [ ]:
# With meshgrids
gto_mesh = GTO(rr_grid, nr_grid, sigma_nr_grid)

# With broadcasting
gto_broadcast = GTO(rr_grid[np.newaxis, :], 
                    np.arange(0, n_max)[:, np.newaxis],
                    GTO_sigma(cutoff, np.arange(0, n_max), n_max)[:, np.newaxis])

np.allclose(gto_mesh, gto_broadcast)

In [ ]:
R_n = np.matmul(S_inv, GTO(rr_grid, nr_grid, sigma_nr_grid))

In [ ]:
nt_grid, tt_grid = np.meshgrid(np.arange(0, l_max+1), t_grid)
nt_grid = nt_grid.T
tt_grid = tt_grid.T

In [ ]:
P_l = legendre_polynomials(nt_grid, tt_grid)

In [ ]:
# Could the non-one-ness and non-zero-ness of the higher orders affect
# the density calculation?
for n in range(0, n_max):
    for m in range(0, n_max):
        print(n, m)
        print(np.sum(R_n[n] * R_n[m]) * dr)

In [ ]:
for i in range(0, n_max):
    plt.plot(r_grid, R_n[i])
    
plt.show()

In [ ]:
for i in range(0, l_max+1):
    plt.plot(t_grid, P_l[i])
    
plt.show()

In [ ]:
feature_idx_map

In [ ]:
n_species = len(frames[0].numbers)
n_pairs = n_species * (n_species + 1) // 2
n_centers = soaps.shape[0]
soaps_reshaped = np.stack([np.reshape(soap, (n_pairs, n_max, n_max, l_max+1)) for soap in soaps])

## Loop (slow!)

In [ ]:
#%%timeit
density = np.zeros((n_pairs, n_centers, len(r_grid), len(r_grid), len(t_grid)))
for p in range(0, n_pairs):
    for n in range(0, n_max):
        for m in range(0, n_max):
            for l in range(0, l_max+1):
                density[p] += R_n[n][np.newaxis, :, np.newaxis, np.newaxis] \
                           * R_n[m][np.newaxis, np.newaxis, :, np.newaxis] \
                           * P_l[l][np.newaxis, np.newaxis, np.newaxis, :] \
                           * soaps_reshaped[:, p, n, m, l][:, np.newaxis, np.newaxis, np.newaxis]

In [ ]:
rx_grid, ry_grid, tz_grid = np.meshgrid(r_grid, r_grid, t_grid)

In [ ]:
fig = go.Figure(data=go.Volume(x=rx_grid.flatten(),
                               y=ry_grid.flatten(),
                               z=tz_grid.flatten(),
                               value=density[1][1].flatten(),
                               isomin=1.0E-4,
                               isomax=None,
                               opacity=0.2,
                               surface_count=20))
fig.show()
#fig.write_html('simple_cubic-n10-l4-c6-gs0.3-cw0.5.html')

## Broadcasting (high memory!)

In [ ]:
#%%timeit
r_n = np.reshape(R_n, (n_max, 1, 1, len(r_grid), 1, 1))
r_m = np.reshape(R_n, (1, n_max, 1, 1, len(r_grid), 1))
p_l = np.reshape(P_l, (1, 1, l_max+1, 1, 1, len(t_grid)))
density = np.tensordot(soaps_reshaped, r_n*r_m*p_l, axes=3)

In [ ]:
rx_grid, ry_grid, tz_grid = np.meshgrid(r_grid, r_grid, t_grid)

In [ ]:
fig = go.Figure(data=go.Volume(x=rx_grid.flatten(),
                               y=ry_grid.flatten(),
                               z=tz_grid.flatten(),
                               value=density[1][1].flatten(),
                               isomin=1.0E-4,
                               isomax=None,
                               opacity=0.2,
                               surface_count=20))
fig.show()
#fig.write_html('simple_cubic-n10-l4-c6-gs0.3-cw0.5.html')

## Broadcasting with grid chunking (compromise!)

In [ ]:
# Can use this if computing R_n and P_l in the chunk loop
chunk_size_r = 12
chunk_size_t = 12
n_chunks_r = len(r_grid) // chunk_size_r
n_chunks_t = len(t_grid) // chunk_size_t
if len(r_grid) % chunk_size_r > 0:
    n_chunks_r += 1
if len(t_grid) % chunk_size_t > 0:
    n_chunks_t += 1
    
r_grid_split = np.array_split(r_grid, n_chunks_r)
t_grid_split = np.array_split(t_grid, n_chunks_t)

In [ ]:
#%%timeit
density = np.zeros((n_centers, n_pairs, len(r_grid), len(r_grid), len(t_grid)))

for n in range(0, n_chunks_r):
    for m in range(0, n_chunks_r):
        for t in range(0, n_chunks_r):
        
            # Can also compute R_n and P_l here in the loop
            slice_n = slice(n * chunk_size_r, (n + 1) * chunk_size_r, 1)
            slice_m = slice(m * chunk_size_r, (m + 1) * chunk_size_r, 1)
            slice_t = slice(t * chunk_size_t, (t + 1) * chunk_size_t, 1)
            r_n = np.reshape(R_n[:, slice_n], (n_max, 1, 1, -1, 1, 1))
            r_m = np.reshape(R_n[:, slice_m], (1, n_max, 1, 1, -1, 1))
            p_l = np.reshape(P_l[:, slice_t], (1, 1, l_max+1, 1, 1, -1))
            density[:, :, slice_n, slice_m, slice_t] = np.tensordot(soaps_reshaped, r_n*r_m*p_l, axes=3)

In [ ]:
rx_grid, ry_grid, tz_grid = np.meshgrid(r_grid, r_grid, t_grid)

In [ ]:
fig = go.Figure(data=go.Volume(x=rx_grid.flatten(),
                               y=ry_grid.flatten(),
                               z=tz_grid.flatten(),
                               value=density[1][1].flatten(),
                               isomin=1.0E-4,
                               isomax=None,
                               opacity=0.2,
                               surface_count=20))
fig.show()
#fig.write_html('simple_cubic-n10-l4-c6-gs0.3-cw0.5.html')